HW4-1 Pycarat to compare ML agorithms on classification problem 16 Model (multi-features titanic is ok , ) 

In [1]:
import pandas as pd
from pycaret.classification import *

# 加載 Titanic 資料
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# ====================
# **特徵工程**
# ====================

# FamilySize 特徵
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

# Title 特徵
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
title_mapping = {
    "Mr": "Mr", "Miss": "Miss", "Mrs": "Mrs",
    "Master": "Master", "Dr": "Rare", "Rev": "Rare",
    "Col": "Rare", "Major": "Rare", "Mlle": "Miss",
    "Countess": "Rare", "Ms": "Miss", "Lady": "Rare",
    "Jonkheer": "Rare", "Don": "Rare", "Dona": "Rare",
    "Mme": "Mrs", "Capt": "Rare", "Sir": "Rare"
}
train['Title'] = train['Title'].map(title_mapping)
test['Title'] = test['Title'].map(title_mapping)

# 填補缺失值
train['Age'].fillna(train.groupby('Title')['Age'].transform('median'), inplace=True)
test['Age'].fillna(test.groupby('Title')['Age'].transform('median'), inplace=True)
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

# 刪除無用特徵
train.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)
test.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

# =====================
# **PyCaret 模型構建**
# =====================

# 初始化 PyCaret 環境
clf_setup = setup(
    data=train,
    target='Survived',
    session_id=123,
    numeric_features=['Age', 'Fare', 'SibSp', 'Parch', 'FamilySize'],
    categorical_features=['Sex', 'Embarked', 'Title']
)

# 比較多種模型，至少 16 個
best_model = compare_models(n_select=16)  # 返回性能最佳的 16 個模型

# ===========================
# **輸出模型比較結果**
# ===========================
# 查看模型比較的性能表格
print("模型比較結果：")
print(pull())  # 提取並顯示比較表格

# ===========================
# **測試數據預測（可選）**
# ===========================
# 使用性能最佳的模型進行測試數據預測
final_model = finalize_model(best_model[0])  # 選取第一個最佳模型
test_predictions = predict_model(final_model, data=test)

# 儲存提交結果
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_predictions['prediction_label']
})
submission.to_csv('submission_hw4_1.csv', index=False)

,Description,Value
0,Session id,123
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 11)"
4,Transformed data shape,"(891, 17)"
5,Transformed train set shape,"(623, 17)"
6,Transformed test set shape,"(268, 17)"
7,Numeric features,5
8,Categorical features,3
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8297,0.8416,0.7362,0.8048,0.7665,0.6334,0.6370,0.0300
lr,Logistic Regression,0.8281,0.8636,0.7571,0.7864,0.7687,0.6324,0.6354,0.3870
ridge,Ridge Classifier,0.8232,0.8637,0.7486,0.7804,0.7617,0.6217,0.6243,0.0140
lda,Linear Discriminant Analysis,0.8232,0.8627,0.7486,0.7804,0.7617,0.6217,0.6243,0.0150
rf,Random Forest Classifier,0.8169,0.8471,0.7279,0.7859,0.7534,0.6083,0.6116,0.0470
nb,Naive Bayes,0.8072,0.8319,0.7402,0.7540,0.7442,0.5898,0.5929,0.0130
lightgbm,Light Gradient Boosting Machine,0.7993,0.8289,0.6694,0.7788,0.7162,0.5632,0.5696,0.0760
ada,Ada Boost Classifier,0.7976,0.8376,0.7239,0.7395,0.7260,0.5670,0.5714,0.0250
et,Extra Trees Classifier,0.7849,0.8366,0.7072,0.7313,0.7150,0.5430,0.5467,0.0390
dt,Decision Tree Classifier,0.7029,0.6936,0.6529,0.6059,0.6245,0.3802,0.3845,0.0140


模型比較結果：
                                    Model  Accuracy     AUC  Recall   Prec.  \
gbc          Gradient Boosting Classifier    0.8297  0.8416  0.7362  0.8048   
lr                    Logistic Regression    0.8281  0.8636  0.7571  0.7864   
ridge                    Ridge Classifier    0.8232  0.8637  0.7486  0.7804   
lda          Linear Discriminant Analysis    0.8232  0.8627  0.7486  0.7804   
rf               Random Forest Classifier    0.8169  0.8471  0.7279  0.7859   
nb                            Naive Bayes    0.8072  0.8319  0.7402  0.7540   
lightgbm  Light Gradient Boosting Machine    0.7993  0.8289  0.6694  0.7788   
ada                  Ada Boost Classifier    0.7976  0.8376  0.7239  0.7395   
et                 Extra Trees Classifier    0.7849  0.8366  0.7072  0.7313   
dt               Decision Tree Classifier    0.7029  0.6936  0.6529  0.6059   
dummy                    Dummy Classifier    0.6164  0.5000  0.0000  0.0000   
knn                K Neighbors Classifier   

HW4-2 對 model optimization using pycarat or optuna or other AutoML,meta-heuristic
  1. Feature engineering, 2. model selection (ensemble), 3 . training 超參數優化

In [2]:
import pandas as pd
from pycaret.classification import *

# 加載 Titanic 資料
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# ====================
# **特徵工程**
# ====================
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

train['IsAlone'] = ((train['SibSp'] + train['Parch']) == 0).astype(int)
test['IsAlone'] = ((test['SibSp'] + test['Parch']) == 0).astype(int)

train['FareBin'] = pd.qcut(train['Fare'], 4, labels=False)
test['FareBin'] = pd.qcut(test['Fare'], 4, labels=False)

train['AgeBin'] = pd.cut(train['Age'], bins=[0, 12, 20, 40, 60, 80], labels=False)
test['AgeBin'] = pd.cut(test['Age'], bins=[0, 12, 20, 40, 60, 80], labels=False)

train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
title_mapping = {
    "Mr": "Mr", "Miss": "Miss", "Mrs": "Mrs", "Master": "Master", "Dr": "Rare", "Rev": "Rare"
}
train['Title'] = train['Title'].map(title_mapping)
test['Title'] = test['Title'].map(title_mapping)

train['Age'].fillna(train.groupby('Title')['Age'].transform('median'), inplace=True)
test['Age'].fillna(test.groupby('Title')['Age'].transform('median'), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

train.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)
test.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

# =====================
# **PyCaret 模型構建**
# =====================
clf_setup = setup(
    data=train,
    target='Survived',
    session_id=123,
    numeric_features=['Age', 'Fare', 'SibSp', 'Parch', 'FamilySize', 'FareBin', 'AgeBin'],
    categorical_features=['Sex', 'Embarked', 'Title'],
    fold_strategy='stratifiedkfold',
    fold=10
)

# 比較模型並選擇最佳模型
best_model = compare_models()

# 調優最佳模型
tuned_model = tune_model(best_model, optimize='AUC', n_iter=100)

# 模型集成
blended_model = blend_models([tuned_model, best_model])
stacked_model = stack_models([tuned_model, best_model])

# 測試數據預測
final_model = finalize_model(stacked_model)
test_predictions = predict_model(final_model, data=test)

# 儲存結果
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_predictions['prediction_label']
})
submission.to_csv('submission_hw4_2.csv', index=False)

,Description,Value
0,Session id,123
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 14)"
4,Transformed data shape,"(891, 20)"
5,Transformed train set shape,"(623, 20)"
6,Transformed test set shape,"(268, 20)"
7,Numeric features,7
8,Categorical features,3
9,Rows with missing values,21.7%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8249,0.8422,0.7277,0.8004,0.7600,0.6230,0.6267,0.0420
lr,Logistic Regression,0.8217,0.8631,0.7487,0.7813,0.7613,0.6195,0.6230,0.0390
gbc,Gradient Boosting Classifier,0.8217,0.8376,0.7237,0.7977,0.7557,0.6163,0.6208,0.0350
ridge,Ridge Classifier,0.8152,0.8644,0.7402,0.7698,0.7522,0.6053,0.6079,0.0130
lda,Linear Discriminant Analysis,0.8152,0.8641,0.7402,0.7698,0.7522,0.6053,0.6079,0.0130
ada,Ada Boost Classifier,0.8088,0.8406,0.7447,0.7567,0.7470,0.5940,0.5974,0.0260
nb,Naive Bayes,0.8056,0.8328,0.7656,0.7391,0.7495,0.5911,0.5944,0.0140
lightgbm,Light Gradient Boosting Machine,0.7945,0.8251,0.6775,0.7639,0.7153,0.5558,0.5605,0.0770
et,Extra Trees Classifier,0.7865,0.8330,0.7071,0.7322,0.7176,0.5463,0.5481,0.0380
dt,Decision Tree Classifier,0.7223,0.7076,0.6446,0.6370,0.6384,0.4135,0.4155,0.0140


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8095,0.8665,0.7083,0.7727,0.7391,0.5896,0.5910
1,0.8413,0.8344,0.7500,0.8182,0.7826,0.6580,0.6595
2,0.9048,0.9530,0.9583,0.8214,0.8846,0.8043,0.8113
3,0.8226,0.8874,0.6522,0.8333,0.7317,0.6021,0.6122
4,0.7742,0.7708,0.6250,0.7500,0.6818,0.5090,0.5141
5,0.7742,0.8070,0.7083,0.7083,0.7083,0.5241,0.5241
6,0.9032,0.8925,0.8750,0.8750,0.8750,0.7961,0.7961
7,0.7258,0.8191,0.7083,0.6296,0.6667,0.4352,0.4373
8,0.7742,0.8454,0.6250,0.7500,0.6818,0.5090,0.5141


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8095,0.8761,0.7500,0.7500,0.7500,0.5962,0.5962
1,0.8571,0.8376,0.7917,0.8261,0.8085,0.6947,0.6951
2,0.8730,0.9316,0.8750,0.8077,0.8400,0.7350,0.7366
3,0.8387,0.8740,0.6522,0.8824,0.7500,0.6349,0.6507
4,0.7742,0.7511,0.6250,0.7500,0.6818,0.5090,0.5141
5,0.7903,0.8015,0.6667,0.7619,0.7111,0.5477,0.5507
6,0.9032,0.8783,0.8750,0.8750,0.8750,0.7961,0.7961
7,0.7419,0.8125,0.7500,0.6429,0.6923,0.4723,0.4765
8,0.7903,0.8542,0.6667,0.7619,0.7111,0.5477,0.5507


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8095,0.8739,0.7083,0.7727,0.7391,0.5896,0.5910
1,0.8571,0.8387,0.7917,0.8261,0.8085,0.6947,0.6951
2,0.8571,0.9359,0.8333,0.8000,0.8163,0.6995,0.6999
3,0.8387,0.8763,0.6522,0.8824,0.7500,0.6349,0.6507
4,0.7742,0.7533,0.6250,0.7500,0.6818,0.5090,0.5141
5,0.7742,0.8037,0.6250,0.7500,0.6818,0.5090,0.5141
6,0.9194,0.8794,0.8750,0.9130,0.8936,0.8287,0.8292
7,0.7419,0.8136,0.7083,0.6538,0.6800,0.4644,0.4654
8,0.7742,0.8542,0.6250,0.7500,0.6818,0.5090,0.5141
